# Cost function: cross entropy
<img src ="Images/lab6_1.png">

<img src ="Images/lab6_2.png">

In [1]:
import tensorflow as tf

x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]] #one-hot 인코딩

X = tf.placeholder("float", [None, 4]) #shape x 데이터는 n개(None), 각 요소는 4개의 값을 가진다.
Y = tf.placeholder("float", [None, 3]) #shape y 데이터는 n개(None), 각 요소는 3개의 값을 가진다.
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight') #가중치의 shape는 들어오는 값 input(4), 나가는 값 output(3)
b = tf.Variable(tf.random_normal([nb_classes]), name='bias') #편향은 출력값과 같다.
#레이어의 개수에 따라 shape가 달라진다.

#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b) #활성함수 소프트맥스 (로짓을 소프트맥스에 넣어 출력값을 구한다.)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1)) #Cross entropy 공식
#Cross entropy를 구하는 공식 = -(정답 레이블 * 출력값의 자연로그의 합) 의 평균
#정답 레이블이 0인 경우(False)에는 (0을 곱하므로) 값이 0이 된다.
#정답 레이블이 1(True)이고, 제대로 예측한 경우에는 log(1)이 0이므로 값이 오차값이 0이 된다. 
#정답 레이블이 1(True)이고, 잘못 예측한 경우에는 log(0)이므로 오차값이 무한대로 커진다.
#결국 예측이 틀렸을 경우에 큰 오차값을 가지게 된다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost) #Gradient Descent로 최적화.

# Launch graph
with tf.Session() as session:
    session.run(tf.global_variables_initializer()) #초기화
    
    for step in range(2001):
        session.run(optimizer, feed_dict={X: x_data, Y: y_data})#모든 텐서가 optimizer로 연결되어 있다.
        if step % 200 == 0:
            print(step, session.run(cost, feed_dict={X: x_data, Y: y_data}))
            
    print('--------------')
    
    # Testing & One-hot encoding
    #학습 종료 후 예측 테스트
    a = session.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, session.run(tf.arg_max(a, 1))) #arg_max 가장 높은 확률의 인덱스를 리턴, 1번째 축

    print('--------------')

    b = session.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, session.run(tf.arg_max(b, 1)))

    print('--------------')

    c = session.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, session.run(tf.arg_max(c, 1)))

    print('--------------')

    all = session.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, session.run(tf.arg_max(all, 1)))


0 8.14539
200 0.645998
400 0.537253
600 0.456427
800 0.385696
1000 0.31692
1200 0.248394
1400 0.221689
1600 0.202485
1800 0.186194
2000 0.172216
--------------
[[  7.15193674e-02   9.28469956e-01   1.07614487e-05]] [1]
--------------
[[ 0.56815344  0.38948864  0.04235791]] [0]
--------------
[[  2.77647896e-08   4.62517812e-04   9.99537468e-01]] [2]
--------------
[[  7.15193674e-02   9.28469956e-01   1.07614487e-05]
 [  5.68153441e-01   3.89488637e-01   4.23579067e-02]
 [  2.77647896e-08   4.62517812e-04   9.99537468e-01]] [1 0 2]


# Fancy Softmax Classifier

<img src="Images/lab6_3.png">

In [2]:
import numpy as np

xy = np.loadtxt("data-04-zoo.csv", delimiter=",", dtype=np.float32)
x_data = xy[:, 0:-1] #인풋 데이터. 전체 행을 가져온다, 각 행의 0 ~ index-1 행까지 가져온다.
y_data = xy[:, [-1]] #정답 레이블. 전체 행을 가져온다, 각 행의 마지막 행만 가져온다.

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16]) #입력. 전체 요소의 개수는 n, 각 요소는 16개씩
Y = tf.placeholder(tf.int32, [None, 1])  #정답레이블. 전체 요소의 개수는 n, 각 요소는 1개씩. shape=(?, 1)
#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot. (입력 데이터, 클래스 수). TF 라이브러리로 구현. shape=(?, 1, 7)
print("one_hot", Y_one_hot)
#tf.one_hot을 실행하면 차원이 하나씩 늘어난다. 
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # (입력 데이터, shape). [None-1, nb_classes]으로 생성. shape=(?, 7)
print("reshape", Y_one_hot)
#reshape를 해야 원하는 모양대로 제대로 one-hot 인코딩이 된다.

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight') #가중치. 입력 16, 출력 7
b = tf.Variable(tf.random_normal([nb_classes]), name='bias') #편향. 출력과 같다 (7)

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b #로짓을 구한다.
hypothesis = tf.nn.softmax(logits) #출력함수는 소프트맥스

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
#TF라이브러리를 이용해 CrossEntropy를 구한다. 이전과 달리 출력함수를 통과한 소프트맥스 출력값이 아닌 로짓을 넣어야 한다. (로짓, 정답 레이블)
cost = tf.reduce_mean(cost_i) #코스트 계산
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost) #최적화.

prediction = tf.argmax(hypothesis, 1) #예측된 값에서 가장 높은 값을 가지는 요소의 인덱스 추출
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #정답레이블과 예상치 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #정확도 계산

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #초기화

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data}) #optimizer에 모든 텐서가 연결되어 있으므로 optimizer 실행
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

    # Let's see if we can predict
    #학습 이후 테스트
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()): #flattend은 1차원으로 변환. [[1], [0]] -> [1, 0]. 
        #zip으로 묶어 각 값을 p와 y로
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


(101, 16) (101, 1)
one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Step:     0	Loss: 6.037	Acc: 5.94%
Step:   100	Loss: 0.586	Acc: 81.19%
Step:   200	Loss: 0.336	Acc: 93.07%
Step:   300	Loss: 0.244	Acc: 95.05%
Step:   400	Loss: 0.195	Acc: 96.04%
Step:   500	Loss: 0.163	Acc: 96.04%
Step:   600	Loss: 0.140	Acc: 98.02%
Step:   700	Loss: 0.123	Acc: 99.01%
Step:   800	Loss: 0.110	Acc: 100.00%
Step:   900	Loss: 0.100	Acc: 100.00%
Step:  1000	Loss: 0.091	Acc: 100.00%
Step:  1100	Loss: 0.084	Acc: 100.00%
Step:  1200	Loss: 0.078	Acc: 100.00%
Step:  1300	Loss: 0.072	Acc: 100.00%
Step:  1400	Loss: 0.068	Acc: 100.00%
Step:  1500	Loss: 0.064	Acc: 100.00%
Step:  1600	Loss: 0.060	Acc: 100.00%
Step:  1700	Loss: 0.057	Acc: 100.00%
Step:  1800	Loss: 0.054	Acc: 100.00%
Step:  1900	Loss: 0.052	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Pr